In [ ]:
import copy
import torch
class ConstantTao:
    def __init__( self, n, initial_tao ):
        self.n = n
        self.tao_reserve = [ initial_tao/n for i in range(self.n) ]  
        self.alpha_reserve = [ initial_tao for i in range(self.n) ]
        self.alpha_outstanding = [ initial_tao for i in range(self.n) ]
        self.k = [ self.tao_reserve[i] * self.alpha_reserve[i] for i in range(n) ]

    def __repr__( self ):
        return self.__str__()

    def __str__( self ):
        return f"tao_reserve: {self.tao_reserve}\ntao_prices: {self.tao_prices()}\nalpha_prices: {self.alpha_prices()}\nalpha_reserve: {self.alpha_reserve}\nalpha_outstanding: {self.alpha_outstanding}\nk: {self.k}"
    
    def __state__dict__( self ):
        return { 
            "tao_reserve": copy.deepcopy( self.tao_reserve ),
            "tao_prices": copy.deepcopy( self.tao_prices() ) ,
            "alpha_prices": copy.deepcopy( self.alpha_prices() ),
            "alpha_reserve": copy.deepcopy( self.alpha_reserve ),
            "alpha_outstanding":copy.deepcopy(  self.alpha_outstanding ),
            "k": copy.deepcopy( self.k ),
            'sum_marketcap': sum( self.marketcaps() ),
            'sum_price': sum( self.alpha_prices() ),
        }

    def tao_prices( self ):
        """ Calculate and return the current price of tao based on reserves. """
        return [ self.alpha_reserve[i] / self.tao_reserve[i] for i in range(len(self.tao_reserve)) ]

    def alpha_prices( self ):
        """ Calculate and return the current price of alpha based on reserves. """
        return [ self.tao_reserve[i] / self.alpha_reserve[i] for i in range(len(self.tao_reserve)) ]
    
    def marketcaps( self ):
        """ Calculate and return the market capitalization of alpha. """
        return [ (self.alpha_outstanding[i] + self.alpha_reserve[i]) * self.alpha_prices()[i] for i in range(len(self.tao_reserve)) ] 

    def block_step( self ):
        for i in range(self.n):
            ratio_i = self.alpha_prices()[i] / sum( self.alpha_prices() )
            self.tao_reserve[i] += 7200 * ratio_i
            self.alpha_reserve[i] += 7200
            self.alpha_outstanding[i] += 7200
            self.k[i] = self.tao_reserve[i] * self.alpha_reserve[i]

    def swap( self, idx_a, idx_b, amount ):

        new_alpha_reserve_a = self.alpha_reserve[idx_a] + amount
        new_tao_reserve_a = self.k[idx_a] / new_alpha_reserve_a
        tao_bought_a = self.tao_reserve[idx_a] - new_tao_reserve_a
        self.alpha_outstanding[idx_a] = self.alpha_outstanding[idx_a] - amount
        self.alpha_reserve[idx_a] = new_alpha_reserve_a
        self.tao_reserve[idx_a] = new_tao_reserve_a
        self.k[idx_a] = self.tao_reserve[idx_a] * self.alpha_reserve[idx_a]

        new_tao_reserve_b = self.tao_reserve[idx_b] + tao_bought_a
        new_alpha_reserve_b = self.k[idx_b] / new_tao_reserve_b
        alpha_bought_b = self.alpha_reserve[idx_b] - new_alpha_reserve_b
        self.alpha_outstanding[idx_b] = self.alpha_outstanding[idx_b] + alpha_bought_b
        self.alpha_reserve[idx_b] = new_alpha_reserve_b
        self.tao_reserve[idx_b] = new_tao_reserve_b
        self.k[idx_b] = self.tao_reserve[idx_b] * self.alpha_reserve[idx_b]